In [1]:
import os
import pandas as pd
import json
from param import MEMOR_ROOT, PARSE_TEXT_BATCH_SIZE, TEXTQUESTION_CSV_PATH, PARSED_DATA_ROOT

In [2]:
from parser.unifiedqa import QuestionCollection, QAMachine

/home/yizhou/anaconda3/envs/mm/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
token_model_name = "allenai/unifiedqa-v2-t5-large-1363200" # you can specify the model size here
model_name = "allenai/unifiedqa-v2-t5-large-1363200"

In [4]:
qm = QAMachine(TEXTQUESTION_CSV_PATH, token_model_name, model_name)

load model......


In [5]:
 # save path
text_answer_folder = os.path.join(PARSED_DATA_ROOT,"memor_text_answers")
if not os.path.exists(text_answer_folder):
    os.mkdir(text_answer_folder)

In [6]:
memor_data = json.load(open(os.path.join(MEMOR_ROOT, "data.json")))

In [7]:
count = 0
from tqdm.auto import tqdm
for video_name in tqdm(memor_data):
    count += 1
    # make folder
    save_folder = os.path.join(text_answer_folder, video_name)
    if not os.path.exists(save_folder):
        os.mkdir(save_folder)
    
    video_info = memor_data[video_name]
    for idx in range(len(video_info["seg_ori_ind"])):
        text = video_info["sentences"][idx]
        
        # load json if exists
        json_path = os.path.join(save_folder, str(idx) + ".json")
        if os.path.exists(json_path):
            sentence_info = json.load(open(json_path, "r"))
        else:
            sentence_info = {
                "text": text,
                "qa":[],
            }
        
        # only parse information sentences
        if len(text) > 15:
            # answer questions
            len_q = len(qm.question_collection.question_list)
            for i in range(0, len_q, PARSE_TEXT_BATCH_SIZE):
                question_list = qm.question_collection.question_list[i: min(i + PARSE_TEXT_BATCH_SIZE, len_q)]
                answer_list = qm.question_collection.answer_list[i: min(i + PARSE_TEXT_BATCH_SIZE, len_q)]
                raw_answer_list = qm.question_collection.raw_answer_list[i: min(i + PARSE_TEXT_BATCH_SIZE, len_q)]

                # perform qa
                answer_choices = qm.predict_multi(text, question_list, answer_list, raw_answer_list)
                for j in range(len(answer_choices)):
                    sentence_info["qa"].append([question_list[j], answer_choices[j]])
        
        with open(json_path, 'w') as outfile:
            json.dump(sentence_info, outfile, indent=4)

        
    # if count > 10:
    #     break